# This notebook are for testing some Linear algebra 


In [22]:
import numpy as np
import scipy.linalg as sl
from scipy.linalg import eigh, qr, svd, inv, det

### Build some matrices for testing

In [43]:
nb=3
H=np.array([[1.68366952, 1.59227351, 1.06408247],
       [1.59227351, 1.60517898, 1.1263274 ],
       [1.06408247, 1.1263274 , 0.15138217]])

S=np.array([[1, 0.1, 0.3],[0.1, 1, 0.05],[0.3, 0.05, 1]])



#### Test orthogonal eigen problem
##### Testing :
$H\Phi = S \Phi  E$ 

$H=  S \Phi E \Phi^{-1}$

$H\Phi \neq E\Phi S$

$\Phi E S \Phi^\dagger = H$

$\Phi^\dagger S \Phi = I$

$\Phi S \Phi^\dagger \neq I$

$S \Phi \Phi^\dagger = I$

$\Phi^{-1} = \Phi^\dagger S$

$(\Phi^{\dagger})^{-1}= S \Phi$

In [45]:
E, Phi =sl.eigh(H, S)
print(np.isclose(H@Phi, S@Phi@np.diag(E)).all())   #H P = S P E
print(np.isclose(S@ Phi @ np.diag(E) @inv(Phi), H).all()) # H = S P E P^{-1}
print(np.isclose(Phi.T.conj() @H@Phi, np.diag(E)).all())        # Pdagger H P = E
print(np.isclose( Phi.T.conj()@S@Phi,np.eye(nb)).all()) # Pdagger S P = I
print(np.isclose( Phi@S@Phi.T.conj(),np.eye(nb)).all()) # P S Pdagger /=I
print(np.isclose( S@Phi@Phi.T.conj(),np.eye(nb)).all()) # S P Pdagger = I
print(np.isclose( inv(Phi), Phi.T.conj()@S).all())      # inv(P) = Pdagger S
print(np.isclose(inv(Phi.T.conj()), S@Phi).all())       # inv(Pdagger) = SP

True
True
True
True
False
True
True
True


### Test Scdmk

In [62]:
def scdm(psiT, ncol):
    """
    select columns for a psiT.
    """
    _Q, _R, piv = qr(psiT, mode='full', pivoting=True)
    cols = piv[:ncol]
    return cols

def get_Amn(psiT):
    U, S, VT= svd(psiT, full_matrices=False)
    Amn=U@VT
    return Amn

def get_wann(Psi, Amn):
    return Psi@Amn

def get_Hwann(E, Amn):
    return Amn.T.conj()@np.diag(E)@Amn

def scdm_newH(E, psi, nwann):
    psi=psi@np.diag([1,1,0.0])
    col=scdm(psi.T.conj(), nwann)
    psi_s=psi[col,:]
    Amn=get_Amn(psi_s.T.conj())
    Wann=get_wann(psi, Amn)
    Hwann= get_Hwann(E, Amn)
    return Amn,Wann, Hwann

In [75]:

E, Phi=eigh(H, np.eye(3))
Amn,Wann, Hwann = scdm_newH(E, Phi, 2)
Ew, Phiw=eigh(Hwann)
print(H)
print(Hwann)

print(E)
print(Ew)

#print(np.isclose(H@Wann, Wann@Amn.T.conj() @np.diag(Ew)@Amn))
#Phiw.T.conj()@Wann.T.conj()@H@Wann@Phiw
Wann[:,0].dot(Wann[:,1])
np.linalg.norm(Wann, axis=0)

[[1.68366952 1.59227351 1.06408247]
 [1.59227351 1.60517898 1.1263274 ]
 [1.06408247 1.1263274  0.15138217]]
[[-0.45908459  0.14115017]
 [ 0.14115017  0.01914427]]
[-0.49763738  0.05769705  3.880171  ]
[-0.49763738  0.05769705]


array([1., 1.])